Vimos como funciona os neurônios artificiais, e como eles são usados para criar redes neurais. Agora, vamos entender como funcionam os perceptrons. o perceptron pega o TLU (Threshold logic unit), as entradas e saídas são números em vez de binárias, e cada conexão de entrada está associada a um peso.TLU calcula a soma ponderada das entradas, aplica uma função degrau a essa somae gera o resultado

### funções degrais:
- heavside = 0 if z < 0, 1 if z >= 1
- sgn (signal) = -1 if z < 0, 0 se z = 0, +1 is z > 0

a fronteira de decisão de cada neurônio de saída é linear, perceptrons não conseguem aprender padrões complexos. Contudo, se as instâncias de treinamento forem linearmente separáveis, o algoritmo convergiria para uma solução, conforme o teorema da convergência do perceptron.

In [9]:
#SKLearn possui uma classe chamada perceptron, que implementa uma rede TLU única:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2,3)]
y = (iris.target == 0).astype(np.int64)

In [6]:
per_clf = Perceptron()
per_clf.fit(X, y)
y_pred = per_clf.predict([[2, 0.5]])

aplicar o perceptron é a mesma coisa de usar o SGDClassifier com parametros -> loss="perceptron", learning_rate="constant", eta0=1, penalty=None

Uma forma de treinar MLPs (Multi-layered percpetrons) é o algoritmo de backpropagation, que funciona basicamente como o gradiente descendente. ele calcula o gradiente passando pela rede (foward pass e back pass), e com isso calcula o erro da rede em relação a cada parâmetro. O cálculo automático dos gradientes se chama autodiff.

## Como funciona o autodiff:
- Ele manipula um mini-batch por vez e realiza o treinamento completo diversas vezes, sendo chamada cada passada de "Época"
- Cada mini-batch é passado para a camada de entrada da rede, enviando para a primeira camada oculta (camadas do meio das MLPs). O algoritmo calcula a saída de todos os neurônios nesta camada (para todas as instâncias do mini-batch). O resultado é passado para a próxima camada, saída é calculada e assim sucessivamente até obter a saída na última camada. Isso é chamado de __Forward Pass__, justamente como fazer predições, porém todos os resultados intermediários são preservados pq são necessários para o __Backward Pass__
- O algoritmo calcula o erro de saída da rede, usando uma função de perda que compara a saída desejada com a saída real e retornando uma medida de erro
- Calcula o quanto cada conexão de saída contribuiu para o erro com a regra de cadeia (chain rule)
- Ele calcula quanto das contribuções de erro veio de cada conexão na camada inferior (camada de entrada), usando a chain rule de novo, trabalhando em backward até atingingir a camada de entrada
- Por último, o algoritmo executa uma etapa do gradiente descendente para ajustar todos os pesos de conexão da rede, utilizando os gradientes de erro que acabou de calcular

Uma alteração importante feita nesse algoritmo é a substituição da função degrau para a função logística

# Implementação com Keras

In [15]:
import tensorflow as tf
keras = tf.keras

In [16]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [17]:
X_train_full.shape

(60000, 28, 28)

In [ ]:
X_val, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0 # slice e escalonamento das características de entrada
y_val, y_train = y_train_full[:5000], y_train_full[5000:]

In [20]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# criando o modelo usando a sequential api
model = keras.models.Sequential() # cria o modelo sequential, o modelo mais simples do Keras feito de uma pilha de camadas sequenciamente conectadas (sequential API)
model.add(keras.layers.Input(shape=(28, 28))) #primeira camada, é uma camada Input onde mostra para o modelo quando o shape do input Como é a primeira camada, deve especificar o Input, que não inclui o tamanho do batch, apenas o formato das instâncias
model.add(keras.layers.Flatten()) # transconverte cada imagem para uma matriz 1D.
model.add(keras.layers.Dense(300, activation='relu')) # camada oculta Dense com 300 neurônios, usando a função ReLU
model.add(keras.layers.Dense(100, activation='relu')) # camada oculta dense com 100 neurônios também usando a ReLU
model.add(keras.layers.Dense(10, activation='softmax')) #camada de saída Dense com 10 neurônios (um por classe) usando a função de ativação softmax

#as classes estão adicionadas sequencialmente para poder explicar o que cada uma faz, mas utilizariamos uma lista dentro do models.Sequential([parametros])

In [27]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 266,610 (1.02 MB)

 Trainable params: 266,610 (1.02 MB)

 Non-trainable params: 0 (0.00 B)